In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np


from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics.pairwise import cosine_distances as cd
from collections import defaultdict
from pyeer.eer_info import get_eer_stats
from sklearn.metrics.pairwise import manhattan_distances as md

from sklearn.metrics.pairwise import cosine_similarity as cs

from pytorch_metric_learning.distances import LpDistance, CosineSimilarity,SNRDistance

def EERf(resutls):
    resutls= np.array(resutls)
    genuine  = resutls[resutls[:, 1] == 1][:, 0]
    impostor = resutls[resutls[:, 1] == 0][:, 0]
    stats_a = get_eer_stats(genuine, impostor)
    return(stats_a.eer,stats_a.fmr100)

def calculate_and_print_averages(y_train, resutls3):
    u, counts = np.unique(y_train, return_counts=True)
    eer_values = []
    ii = 0

    for i in resutls3.keys():
        re = EERf(resutls3[i])
        eer = re[0]
        print(f"{i}: EER = {re[0]:.4f}, FMR100 = {re[1]:.4f}, Count = {counts[ii]}")
        eer_values.append(eer)
        ii += 1

    average_eer = np.mean(eer_values) * 100
    std_eer = np.std(eer_values) * 100
    
    print(f"Final Average EER: {average_eer:.4f}")
    print(f"Final EER Standard Deviation: {std_eer:.4f}")
    print(f"${average_eer:.2f} \\pm {std_eer:.2f}$")
    return(average_eer,std_eer)

import torch.nn.functional as F





def calculate_similarity_scores_two(enrollment_embeddings, y_enrollment, verification_embeddings, y_verification,distance):
    similarity_results = []
    similarity_results_by_class = []
    similarity_results_by_class_dict = defaultdict(list)
    unique_classes = np.unique(y_enrollment)
    class_indices = [np.where(y_enrollment == cls)[0] for cls in unique_classes]

    if distance == "cd":
        similarity_matrix = -1 * cd(verification_embeddings, enrollment_embeddings)
    elif distance == "ed":
        similarity_matrix = -1 * ed(verification_embeddings, enrollment_embeddings)

    for i in range(similarity_matrix.shape[0]):
        current_class = y_verification[i]
        predicted_scores = similarity_matrix[i]
        same_class_indices = class_indices[np.where(unique_classes == current_class)[0][0]]

        for cls in unique_classes:
            same_class_indices = class_indices[np.where(unique_classes == cls)[0][0]]
            max_score = sum(sorted(predicted_scores[same_class_indices], reverse=True)[:10]) / 10
            if current_class == cls:
                similarity_results_by_class.append([max_score, 1, current_class, cls, i])
                similarity_results_by_class_dict[cls].append([max_score, 1, current_class, cls, i, cls])
            else:
                similarity_results_by_class.append([max_score, 0, current_class, cls, i])
                similarity_results_by_class_dict[cls].append([max_score, 0, current_class, cls, i, cls])

    return similarity_results_by_class, similarity_results_by_class_dict

def assessment_model_data_two(enrollment_data, ye, verification_data, yv, e_network, distance):
    x_enrollment, y_enrollment = enrollment_data, ye
    x_verification, y_verification = verification_data, yv
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    similarity_results_by_class, similarity_results_by_class_dict = calculate_similarity_scores_two(
        enrollment_embeddings, y_enrollment, verification_embeddings, y_verification,distance
    )
    return similarity_results_by_class, similarity_results_by_class_dict




import torch
import numpy as np

def compute_embedding_batch_two(x_test_batch, embedding_network, batch_size=150, device="cuda"):
    total_samples = len(x_test_batch)
    embeddings = []

    # Process the data in batches
    for start_index in range(0, total_samples, batch_size):
        end_index = min(start_index + batch_size, total_samples)

        batch = x_test_batch[start_index:end_index].copy()
        # Convert the current batch to tensor and move to device
        batch = torch.tensor(batch, dtype=torch.float32).to(device)

        with torch.no_grad():  # Disable gradient computation for inference
            batch_embeddings = embedding_network(batch).detach().cpu().numpy()

        embeddings.append(batch_embeddings)  # Store embeddings on CPU

        # Clear GPU memory for the batch
        del batch
        torch.cuda.empty_cache()

    # Concatenate all embeddings into a single numpy array
    anchor_embeddings = np.concatenate(embeddings, axis=0)

    print(f"Computed embeddings shape: {anchor_embeddings.shape}")
    return anchor_embeddings

In [4]:
import torch
import torch.nn as nn

class EEGNet7(nn.Module):
    def __init__(self):
        super(EEGNet7, self).__init__()
        # InstanceNorm1d normalizes each channel across its 500 values for each sample
        self.norm = nn.InstanceNorm1d(14, affine=False)  # Normalizes across the 500 values in each of the 93 channels
        self.act = nn.ReLU()  # or nn.SELU() , ReLU
        self.act2 = nn.Tanh()
        self.conv1 = nn.Conv2d(1, 256, (1, 4), padding='same')
        self.pool1 = nn.MaxPool2d((1, 2))
        self.conv2 = nn.Conv2d(256, 192, (4, 1), padding='same')
        self.pool2 = nn.MaxPool2d((2, 1))
        self.conv3 = nn.Conv2d(192, 128, (1, 4), padding='same')
        #self.pool3 = nn.MaxPool2d((1, 2))
        self.conv4 = nn.Conv2d(128, 96, (4, 1), padding='same')
        #self.pool4 = nn.MaxPool2d((2, 1))
        self.conv5 = nn.Conv2d(96, 64, (1, 4), padding='same')
        #self.pool5 = nn.MaxPool2d((1, 2))
        self.conv6 = nn.Conv2d(64, 32, (4, 1), padding='same')
        #self.pool6 = nn.MaxPool2d((2, 1))
        self.conv7 = nn.Conv2d(32, 16, (1, 2), padding='same')
        self.conv8 = nn.Conv2d(16, 2, (2, 1), padding='same')
        self.flatten = nn.Flatten()
        self.embedding = nn.Linear(1500, 128)  # Embedding layer

    def forward(self, x):
        #if x.dim() == 3:
            #x = x.unsqueeze(1)  # Adding a channel dimension if input is 3D
        x = self.norm(x)  # Apply InstanceNorm along the channel dimension (squeeze to 1D first)
        x = x.unsqueeze(1)  # Reshape to 4D again for Conv2d
        x = self.act(self.conv1(x))
        x = self.pool1(x)
        x = self.act(self.conv2(x))
        x = self.pool2(x)
        x = self.act(self.conv3(x))
        #x = self.pool3(x)
        x = self.act(self.conv4(x))
        #x = self.pool4(x)
        x = self.act(self.conv5(x))
        #x = self.pool5(x)
        x = self.act(self.conv6(x))
        #x = self.pool6(x)
        x = self.act(self.conv7(x))
        x = self.act(self.conv8(x))
        x = self.flatten(x)
        x = self.embedding(x)
        return x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trunk = EEGNet7().to(device)

In [5]:
# import h5py
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, Subset
import h5py
import numpy as np

import h5py

import h5py
import numpy as np

# Load test data from test_raw.h5
with h5py.File("./data/test_raw.h5", "r") as f_test:
    X_test = f_test['data'][:]
    Y_test = f_test['labels'][:]
    S_test = f_test['sessions'][:]
    H_test = f_test['hardwares'][:]  # Load the 'hardwares' dataset as h_test

# Load negative data from neg_raw.h5
with h5py.File("./data/neg_raw.h5", "r") as f_neg:
    X_neg = f_neg['data'][:]
    Y_neg = f_neg['labels'][:]
    S_neg = f_neg['sessions'][:]
    H_neg = f_neg['hardwares'][:]  # Load the 'hardwares' dataset as h_neg

# Optional: Combine or use them independently depending on your requirement
# Example of concatenating them for a combined dataset
X_test = np.concatenate((X_test, X_neg), axis=0)
Y_test = np.concatenate((Y_test, Y_neg), axis=0)
S_test = np.concatenate((S_test, S_neg), axis=0)
H_test = np.concatenate((H_test, H_neg), axis=0)

headset_indices= [30, 62, 59, 63, 72, 78, 86]
X_test = X_test[:, headset_indices, :]
# Now, X_combined, Y_combined, S_combined, and h_combined contain the merged data


# Now, X_combined, Y_combined, and S_combined contain the merged data


# Shuffle the data
indices = np.arange(X_test.shape[0])
np.random.shuffle(indices)

# Reorder the arrays according to the shuffled indices
x_test = X_test[indices]
y_test = Y_test[indices]
s_test = S_test[indices]
h_test = H_test[indices]


# Find unique subjects
unique_subjects = np.unique(y_test)

# Initialize lists to hold the data for x_test_e and x_test_v
x_test_e_list = []
y_test_e_list = []
s_test_e_list = []
h_test_e_list = []

x_test_v_list = []
y_test_v_list = []
s_test_v_list = []
h_test_v_list = []

# Assign the minimum session for each subject to x_test_e and the rest to x_test_v
for subject in unique_subjects:
    subject_indices = np.where(y_test == subject)[0]
    subject_sessions = s_test[subject_indices]
    
    # Skip subjects with fewer than two unique sessions
    if len(np.unique(subject_sessions)) < 2:
        continue

    print(f"Subject {subject}, Number of unique sessions: {len(np.unique(subject_sessions))}")
    
    # Assign the minimum session to the evaluation set (x_test_e)
    min_session = np.min(subject_sessions)
    
    # Append data to the evaluation set (min session)
    x_test_e_list.extend(x_test[subject_indices][subject_sessions == min_session])
    y_test_e_list.extend(y_test[subject_indices][subject_sessions == min_session])
    s_test_e_list.extend(s_test[subject_indices][subject_sessions == min_session])
    h_test_e_list.extend(h_test[subject_indices][subject_sessions == min_session])

    # Append remaining sessions to the validation set (x_test_v)
    x_test_v_list.extend(x_test[subject_indices][subject_sessions != min_session])
    y_test_v_list.extend(y_test[subject_indices][subject_sessions != min_session])
    s_test_v_list.extend(s_test[subject_indices][subject_sessions != min_session])
    h_test_v_list.extend(h_test[subject_indices][subject_sessions != min_session])

# Shuffle and convert lists back to numpy arrays for x_test_e
indices_e = np.arange(len(x_test_e_list))
np.random.shuffle(indices_e)

x_test_e = np.array(x_test_e_list)[indices_e]
y_test_e = np.array(y_test_e_list)[indices_e]
s_test_e = np.array(s_test_e_list)[indices_e]
h_test_e = np.array(h_test_v_list)[indices_e]

# Shuffle and convert lists back to numpy arrays for x_test_v
indices_v = np.arange(len(x_test_v_list))
np.random.shuffle(indices_v)

x_test_v = np.array(x_test_v_list)[indices_v]
y_test_v = np.array(y_test_v_list)[indices_v]
s_test_v = np.array(s_test_v_list)[indices_v]
h_test_v = np.array(h_test_v_list)[indices_v]
# Optional: Save the new test evaluation and validation sets to npy files (if needed)








print(f"x_test_e: {x_test_e.shape}, y_test_e: {y_test_e.shape}, s_test_e: {s_test_e.shape}")
print(f"x_test_v: {x_test_v.shape}, y_test_v: {y_test_v.shape}, s_test_v: {s_test_v.shape}")


# Load the state dictionary from the saved file
state_dict = torch.load('./model/SupConLossLoss128_m4_e99_DSIVR300.pth')

# Load the state dictionary into the model
trunk.load_state_dict(state_dict)

Subject 70, Number of unique sessions: 19
Subject 82, Number of unique sessions: 19
Subject 85, Number of unique sessions: 19
Subject 86, Number of unique sessions: 19
Subject 96, Number of unique sessions: 18
Subject 103, Number of unique sessions: 18
Subject 106, Number of unique sessions: 41
Subject 109, Number of unique sessions: 5
Subject 111, Number of unique sessions: 19
Subject 115, Number of unique sessions: 41
Subject 118, Number of unique sessions: 19
Subject 119, Number of unique sessions: 8
Subject 125, Number of unique sessions: 19
Subject 129, Number of unique sessions: 5
Subject 130, Number of unique sessions: 19
Subject 131, Number of unique sessions: 19
Subject 136, Number of unique sessions: 19
Subject 145, Number of unique sessions: 19
Subject 156, Number of unique sessions: 6
Subject 159, Number of unique sessions: 19
Subject 167, Number of unique sessions: 2
Subject 169, Number of unique sessions: 6
Subject 173, Number of unique sessions: 6
Subject 174, Number of 

/tmp/ipykernel_2016154/921074870.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./model/SupConLossLoss128_m4_e99_DSIVR300.pth')


<All keys matched successfully>

In [6]:
x_test_ee = compute_embedding_batch_two(x_test_e, trunk)
x_test_ve = compute_embedding_batch_two(x_test_v, trunk)

/home/fallahi/anaconda3/lib/python3.12/site-packages/torch/nn/modules/instancenorm.py:88: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
/home/fallahi/anaconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:454: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1031.)
  return F.conv2d(input, weight, bias, self.stride,


Computed embeddings shape: (10000, 128)
Computed embeddings shape: (160051, 128)


In [7]:
#HEREAA
def compute_embedding_batch_two(x_test_batch, embedding_network, batch_size=100, device="cuda"):
    #print(x_test_batch.shape)
    return x_test_batch



# Generate a sequence of indices from 0 to the length of one of the arrays
indices = np.arange(x_test.shape[0])


print(x_test.shape,y_test_v.shape, y_test_e.shape)

x_test_b = x_test_v
y_test_b = y_test_v
s_test_b = s_test_v


#, s_test_b
resutls2,resutls3=assessment_model_data_two(x_test_ee,y_test_e, x_test_ve,y_test_b,trunk, distance = "cd")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

(170051, 7, 500) (160051,) (10000,)
70: EER = 0.2625, FMR100 = 0.9156, Count = 100
82: EER = 0.1226, FMR100 = 0.6194, Count = 100
85: EER = 0.1297, FMR100 = 0.6917, Count = 100
86: EER = 0.0933, FMR100 = 0.5511, Count = 100
96: EER = 0.1512, FMR100 = 0.7282, Count = 100
103: EER = 0.0753, FMR100 = 0.4906, Count = 100
106: EER = 0.3276, FMR100 = 0.8646, Count = 100
109: EER = 0.3262, FMR100 = 0.9525, Count = 100
111: EER = 0.2756, FMR100 = 0.9339, Count = 100
115: EER = 0.3137, FMR100 = 0.8110, Count = 100
118: EER = 0.1392, FMR100 = 0.7506, Count = 100
119: EER = 0.1359, FMR100 = 0.7629, Count = 100
125: EER = 0.1140, FMR100 = 0.7239, Count = 100
129: EER = 0.3151, FMR100 = 0.9175, Count = 100
130: EER = 0.1456, FMR100 = 0.6678, Count = 100
131: EER = 0.1862, FMR100 = 0.8961, Count = 100
136: EER = 0.1380, FMR100 = 0.7872, Count = 100
145: EER = 0.1533, FMR100 = 0.6911, Count = 100
156: EER = 0.0945, FMR100 = 0.5540, Count = 100
159: EER = 0.1674, FMR100 = 0.8389, Count = 100
167: EER 

(16.548069643114346, 7.336914445939547)

In [8]:
def calculate_similarity_scores_two2(enrollment_embeddings,y_enrollment, verification_embeddings, y_verification,s_ver, extra= 3):
    flag_s=0
    similarity_results = []
    similarity_results_by_class = []
    similarity_results_by_class_dict = defaultdict(list)
    unique_classes = np.unique(y_enrollment)
    print(unique_classes)
    class_indices = [np.where(y_enrollment == cls)[0] for cls in unique_classes]
    # Compute similarity matrix
    print("I am 9")
    #similarity_matrix =batch_pairwise_distances(verification_embeddings, enrollment_embeddings, 100)

    print(verification_embeddings.shape, enrollment_embeddings.shape )
    
    similarity_matrix = -1 * cd(verification_embeddings, enrollment_embeddings)
    #similarity_matrix = abs(cs(verification_embeddings, enrollment_embeddings))
    print("I am 10")
    print(similarity_matrix.shape)
    for i in range(similarity_matrix.shape[0]):
        current_class = y_verification[i]
        subject_session = s_ver[i]
        #current_ss= np.where((s_ver == subject_session) & (y_verification == current_class))
        
        current_ss = np.where((s_ver == subject_session) & (y_verification == current_class))[0]  # assuming s_ver == subject_session is an array of booleans
        #print("aa", len(current_ss))
        
        selected_indices = np.random.choice(current_ss, extra, replace=True)
        selected_indices = np.append(selected_indices, i)

        #print(current_class, y_verification[selected_indices], s_ver[selected_indices], selected_indices)

        predicted_scoresg = similarity_matrix[selected_indices]
        


        
        predicted_scores = similarity_matrix[i]


        same_class_indices = class_indices[np.where(unique_classes == current_class)[0][0]]


        for cls in unique_classes:
            same_class_indices = class_indices[np.where(unique_classes == cls)[0][0]]
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #max_score = np.mean(predicted_scores[same_class_indices])
            #max_score = max(predicted_scores[same_class_indices])
            maxscore=[]
            for ir in range(extra+1):
                max_score = sum(sorted(predicted_scoresg[ir, same_class_indices], reverse=True)[:10]) / 10
                maxscore.append(max_score)
            max_score = sum(maxscore)/(extra+1)
            
            if current_class == cls:
                similarity_results_by_class.append([max_score, 1, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 1, current_class, cls,i, cls ])
            else:
                similarity_results_by_class.append([max_score, 0, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 0, current_class, cls,i, cls])

    return similarity_results_by_class, similarity_results_by_class_dict

def assessment_model_data_two2(enrollment_data, ye, verification_data,yv, e_network, s_ver, distance = "cd"):
    x_enrollment, y_enrollment = enrollment_data , ye
    x_verification, y_verification = verification_data, yv


    print(f"Total number of enrollment samples: {len(x_enrollment)}")
    print(f"Total number of verification samples: {len(x_verification)}", verification_data.shape)

    # Compute embeddings for enrollment data
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    print("I am 0 - enrollment", enrollment_embeddings.shape)
    # Compute embeddings for verification data
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    print("I am 1")

    similarity_results_by_class, similarity_results_by_class_dict = calculate_similarity_scores_two2(
        enrollment_embeddings,y_enrollment, verification_embeddings, y_verification, s_ver
    )

    return similarity_results_by_class, similarity_results_by_class_dict

In [9]:

#, s_test_b
resutls2,resutls3=assessment_model_data_two2(x_test_ee,y_test_e, x_test_ve,y_test_v,trunk, s_test_v, distance = "cd")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

Total number of enrollment samples: 10000
Total number of verification samples: 160051 (160051, 128)
I am 0 - enrollment (10000, 128)
I am 1
[ 70  82  85  86  96 103 106 109 111 115 118 119 125 129 130 131 136 145
 156 159 167 169 173 174 175 176 181 183 184 187 191 192 194 197 199 201
 203 206 207 214 216 217 219 221 225 230 231 233 236 245 249 250 251 257
 260 261 262 265 268 270 277 281 282 284 286 291 296 299 301 303 306 310
 314 322 325 332 336 342 348 354 357 364 365 373 374 384 385 387 402 404
 407 413 414 415 418 419 422 423 433 436]
I am 9
(160051, 128) (10000, 128)
I am 10
(160051, 10000)
70: EER = 0.2119, FMR100 = 0.8683, Count = 100
82: EER = 0.0855, FMR100 = 0.3923, Count = 100
85: EER = 0.0885, FMR100 = 0.5728, Count = 100
86: EER = 0.0575, FMR100 = 0.3317, Count = 100
96: EER = 0.0953, FMR100 = 0.5541, Count = 100
103: EER = 0.0335, FMR100 = 0.1865, Count = 100
106: EER = 0.3169, FMR100 = 0.8120, Count = 100
109: EER = 0.3311, FMR100 = 0.9800, Count = 100
111: EER = 0.24

(12.703086967759905, 8.1558837504068)

In [10]:

X_test = np.concatenate((x_test_ee, x_test_ve), axis=0)
Y_test = np.concatenate((y_test_e, y_test_v), axis=0)
S_test = np.concatenate((s_test_e, s_test_v), axis=0)
H_test = np.concatenate((h_test_e, h_test_v), axis=0)

In [17]:



import numpy as np

#uy= np.unique(Y_test)[:5]
#ynl = np.isin(Y_test,uy)


#X_test = X_test[ynl]
#Y_test = Y_test[ynl]
#S_test = S_test[ynl]

# Generate a sequence of indices from 0 to the length of one of the arrays
indices = np.arange(X_test.shape[0])

# Shuffle the indices
np.random.shuffle(indices)

# Use the shuffled indices to reorder the arrays
x_test = X_test[indices]
y_test = Y_test[indices]
s_test = S_test[indices]

# Find unique subjects
unique_subjects = np.unique(y_test)

# Initialize lists to hold the data for x_test_e and x_test_v
x_test_e_list = []
y_test_e_list = []
s_test_e_list = []

x_test_v_list = []
y_test_v_list = []
s_test_v_list = []

# Assign the minimum session for each subject to x_test_e and the rest to x_test_v
# Define the number of sessions for enrollment and verification
num_enrollment_sessions = 2
num_verification_sessions = 1  # This can be adjusted as needed

for subject in unique_subjects:
    subject_indices = np.where(y_test == subject)[0]
    subject_sessions = s_test[subject_indices]
    unique_sessions = np.unique(subject_sessions)
    
    # Ensure there are enough sessions available for both enrollment and verification
    if len(unique_sessions) < (num_enrollment_sessions + num_verification_sessions):
        continue

    #print(subject, len(unique_sessions))
    
    # Select sessions for enrollment and verification
    enrollment_sessions = unique_sessions[:num_enrollment_sessions]
    verification_sessions = unique_sessions[num_enrollment_sessions:num_enrollment_sessions + num_verification_sessions]
    
    # Extend the enrollment lists with data from selected sessions
    x_test_e_list.extend(x_test[subject_indices][np.isin(subject_sessions, enrollment_sessions)])
    y_test_e_list.extend(y_test[subject_indices][np.isin(subject_sessions, enrollment_sessions)])
    s_test_e_list.extend(s_test[subject_indices][np.isin(subject_sessions, enrollment_sessions)])

    # Extend the verification lists with data from selected sessions
    x_test_v_list.extend(x_test[subject_indices][np.isin(subject_sessions, verification_sessions)])
    y_test_v_list.extend(y_test[subject_indices][np.isin(subject_sessions, verification_sessions)])
    s_test_v_list.extend(s_test[subject_indices][np.isin(subject_sessions, verification_sessions)])


indices = np.arange(np.array(x_test_e_list).shape[0])

# Shuffle the indices
np.random.shuffle(indices)

# Use the shuffled indices to reorder the arrays


# Convert lists back to numpy arrays
x_test_e = np.array(x_test_e_list)
y_test_e = np.array(y_test_e_list)
s_test_e = np.array(s_test_e_list)

x_test_e = x_test_e[indices]
y_test_e = y_test_e[indices]
s_test_e = s_test_e[indices]


indices = np.arange(np.array(x_test_v_list).shape[0])

# Shuffle the indices
np.random.shuffle(indices)


x_test_v = np.array(x_test_v_list)
y_test_v = np.array(y_test_v_list)
s_test_v = np.array(s_test_v_list)

# Use the shuffled indices to reorder the arrays
x_test_v = x_test_v[indices]
y_test_v = y_test_v[indices]
s_test_v = s_test_v[indices]

print("subjects:", len(np.unique(y_test_e)))
print(f"x_test_e: {x_test_e.shape}, y_test_e: {y_test_e.shape}, s_test_e: {s_test_e.shape}")
print(f"x_test_v: {x_test_v.shape}, y_test_v: {y_test_v.shape}, s_test_v: {s_test_v.shape}")

subjects: 99
x_test_e: (19800, 128), y_test_e: (19800,), s_test_e: (19800,)
x_test_v: (9900, 128), y_test_v: (9900,), s_test_v: (9900,)


In [18]:
def calculate_similarity_scores_two2(enrollment_embeddings,y_enrollment, verification_embeddings, y_verification,s_ver, extra= 0):
    flag_s=0
    similarity_results = []
    similarity_results_by_class = []
    similarity_results_by_class_dict = defaultdict(list)
    unique_classes = np.unique(y_enrollment)
    print(unique_classes)
    class_indices = [np.where(y_enrollment == cls)[0] for cls in unique_classes]
    # Compute similarity matrix
    print("I am 9")
    #similarity_matrix =batch_pairwise_distances(verification_embeddings, enrollment_embeddings, 100)

    print(verification_embeddings.shape, enrollment_embeddings.shape )
    
    similarity_matrix = -1 * cd(verification_embeddings, enrollment_embeddings)
    #similarity_matrix = abs(cs(verification_embeddings, enrollment_embeddings))
    print("I am 10")
    print(similarity_matrix.shape)
    for i in range(similarity_matrix.shape[0]):
        current_class = y_verification[i]
        subject_session = s_ver[i]
        #current_ss= np.where((s_ver == subject_session) & (y_verification == current_class))
        
        current_ss = np.where((s_ver == subject_session) & (y_verification == current_class))[0]  # assuming s_ver == subject_session is an array of booleans
        #print("aa", len(current_ss))
        
        selected_indices = np.random.choice(current_ss, extra, replace=True)
        selected_indices = np.append(selected_indices, i)

        #print(current_class, y_verification[selected_indices], s_ver[selected_indices], selected_indices)

        predicted_scoresg = similarity_matrix[selected_indices]
        


        
        predicted_scores = similarity_matrix[i]


        same_class_indices = class_indices[np.where(unique_classes == current_class)[0][0]]


        for cls in unique_classes:
            same_class_indices = class_indices[np.where(unique_classes == cls)[0][0]]
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #max_score = np.mean(predicted_scores[same_class_indices])
            #max_score = max(predicted_scores[same_class_indices])
            maxscore=[]
            for ir in range(extra+1):
                max_score = sum(sorted(predicted_scoresg[ir, same_class_indices], reverse=True)[:10]) / 10
                maxscore.append(max_score)
            max_score = sum(maxscore)/(extra+1)
            
            if current_class == cls:
                similarity_results_by_class.append([max_score, 1, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 1, current_class, cls,i, cls ])
            else:
                similarity_results_by_class.append([max_score, 0, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 0, current_class, cls,i, cls])

    return similarity_results_by_class, similarity_results_by_class_dict

def assessment_model_data_two2(enrollment_data, ye, verification_data,yv, e_network, s_ver, distance = "cd"):
    x_enrollment, y_enrollment = enrollment_data , ye
    x_verification, y_verification = verification_data, yv


    print(f"Total number of enrollment samples: {len(x_enrollment)}")
    print(f"Total number of verification samples: {len(x_verification)}", verification_data.shape)

    # Compute embeddings for enrollment data
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    print("I am 0 - enrollment", enrollment_embeddings.shape)
    # Compute embeddings for verification data
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    print("I am 1")

    similarity_results_by_class, similarity_results_by_class_dict = calculate_similarity_scores_two2(
        enrollment_embeddings,y_enrollment, verification_embeddings, y_verification, s_ver
    )

    return similarity_results_by_class, similarity_results_by_class_dict

In [19]:

#, s_test_b
resutls2,resutls3=assessment_model_data_two2(x_test_e,y_test_e, x_test_v,y_test_v,None, s_test_v, distance = "cd")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

Total number of enrollment samples: 19800
Total number of verification samples: 9900 (9900, 128)
I am 0 - enrollment (19800, 128)
I am 1
[ 70  82  85  86  96 103 106 109 111 115 118 119 125 129 130 131 136 145
 156 159 169 173 174 175 176 181 183 184 187 191 192 194 197 199 201 203
 206 207 214 216 217 219 221 225 230 231 233 236 245 249 250 251 257 260
 261 262 265 268 270 277 281 282 284 286 291 296 299 301 303 306 310 314
 322 325 332 336 342 348 354 357 364 365 373 374 384 385 387 402 404 407
 413 414 415 418 419 422 423 433 436]
I am 9
(9900, 128) (19800, 128)
I am 10
(9900, 19800)
70: EER = 0.1230, FMR100 = 0.7400, Count = 200
82: EER = 0.0321, FMR100 = 0.3300, Count = 200
85: EER = 0.0700, FMR100 = 0.7100, Count = 200
86: EER = 0.0149, FMR100 = 0.0800, Count = 200
96: EER = 0.1000, FMR100 = 0.6800, Count = 200
103: EER = 0.0416, FMR100 = 0.4100, Count = 200
106: EER = 0.1107, FMR100 = 0.8100, Count = 200
109: EER = 0.0900, FMR100 = 0.3500, Count = 200
111: EER = 0.2300, FMR100 =

(11.942949907235622, 7.044733163868219)

In [20]:
def calculate_similarity_scores_two2(enrollment_embeddings,y_enrollment, verification_embeddings, y_verification,s_ver, extra= 3):
    flag_s=0
    similarity_results = []
    similarity_results_by_class = []
    similarity_results_by_class_dict = defaultdict(list)
    unique_classes = np.unique(y_enrollment)
    print(unique_classes)
    class_indices = [np.where(y_enrollment == cls)[0] for cls in unique_classes]
    # Compute similarity matrix
    print("I am 9")
    #similarity_matrix =batch_pairwise_distances(verification_embeddings, enrollment_embeddings, 100)

    print(verification_embeddings.shape, enrollment_embeddings.shape )
    
    similarity_matrix = -1 * cd(verification_embeddings, enrollment_embeddings)
    #similarity_matrix = abs(cs(verification_embeddings, enrollment_embeddings))
    print("I am 10")
    print(similarity_matrix.shape)
    for i in range(similarity_matrix.shape[0]):
        current_class = y_verification[i]
        subject_session = s_ver[i]
        #current_ss= np.where((s_ver == subject_session) & (y_verification == current_class))
        
        current_ss = np.where((s_ver == subject_session) & (y_verification == current_class))[0]  # assuming s_ver == subject_session is an array of booleans
        #print("aa", len(current_ss))
        
        selected_indices = np.random.choice(current_ss, extra, replace=True)
        selected_indices = np.append(selected_indices, i)

        #print(current_class, y_verification[selected_indices], s_ver[selected_indices], selected_indices)

        predicted_scoresg = similarity_matrix[selected_indices]
        


        
        predicted_scores = similarity_matrix[i]


        same_class_indices = class_indices[np.where(unique_classes == current_class)[0][0]]


        for cls in unique_classes:
            same_class_indices = class_indices[np.where(unique_classes == cls)[0][0]]
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #max_score = np.mean(predicted_scores[same_class_indices])
            #max_score = max(predicted_scores[same_class_indices])
            maxscore=[]
            for ir in range(extra+1):
                max_score = sum(sorted(predicted_scoresg[ir, same_class_indices], reverse=True)[:10]) / 10
                maxscore.append(max_score)
            max_score = sum(maxscore)/(extra+1)
            
            if current_class == cls:
                similarity_results_by_class.append([max_score, 1, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 1, current_class, cls,i, cls ])
            else:
                similarity_results_by_class.append([max_score, 0, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 0, current_class, cls,i, cls])

    return similarity_results_by_class, similarity_results_by_class_dict

def assessment_model_data_two2(enrollment_data, ye, verification_data,yv, e_network, s_ver, distance = "cd"):
    x_enrollment, y_enrollment = enrollment_data , ye
    x_verification, y_verification = verification_data, yv


    print(f"Total number of enrollment samples: {len(x_enrollment)}")
    print(f"Total number of verification samples: {len(x_verification)}", verification_data.shape)

    # Compute embeddings for enrollment data
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    print("I am 0 - enrollment", enrollment_embeddings.shape)
    # Compute embeddings for verification data
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    print("I am 1")

    similarity_results_by_class, similarity_results_by_class_dict = calculate_similarity_scores_two2(
        enrollment_embeddings,y_enrollment, verification_embeddings, y_verification, s_ver
    )

    return similarity_results_by_class, similarity_results_by_class_dict

In [21]:

#, s_test_b
resutls2,resutls3=assessment_model_data_two2(x_test_e,y_test_e, x_test_v,y_test_v,None, s_test_v, distance = "cd")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

Total number of enrollment samples: 19800
Total number of verification samples: 9900 (9900, 128)
I am 0 - enrollment (19800, 128)
I am 1
[ 70  82  85  86  96 103 106 109 111 115 118 119 125 129 130 131 136 145
 156 159 169 173 174 175 176 181 183 184 187 191 192 194 197 199 201 203
 206 207 214 216 217 219 221 225 230 231 233 236 245 249 250 251 257 260
 261 262 265 268 270 277 281 282 284 286 291 296 299 301 303 306 310 314
 322 325 332 336 342 348 354 357 364 365 373 374 384 385 387 402 404 407
 413 414 415 418 419 422 423 433 436]
I am 9
(9900, 128) (19800, 128)
I am 10
(9900, 19800)
70: EER = 0.0602, FMR100 = 0.4700, Count = 200
82: EER = 0.0047, FMR100 = 0.0000, Count = 200
85: EER = 0.0434, FMR100 = 0.5700, Count = 200
86: EER = 0.0010, FMR100 = 0.0000, Count = 200
96: EER = 0.0610, FMR100 = 0.3400, Count = 200
103: EER = 0.0042, FMR100 = 0.0000, Count = 200
106: EER = 0.1007, FMR100 = 0.7200, Count = 200
109: EER = 0.0237, FMR100 = 0.0600, Count = 200
111: EER = 0.1308, FMR100 =

(6.865749330035044, 6.966859200985594)